### Gera Coleção Dourada (.json) e planilha de métricas Kappa - Última versão
Última atualização em 16/12 - Geração da CDJUR - TJ após 2a. revisão

#### Função para checar se há interseção entre dois intervalos

In [1]:
def getOverlap(a, b):
    overlap = max(0, min(a[1], b[1]) - max(a[0], b[0]))
    if overlap > 0:
        if a[0] < b[0]:
            return True #, 0
        else:
            return True # , 1
    else:
        return False

# inte = getOverlap([565, 667],[530, 567])

# inte =  getOverlap([10, 15], [20, 38])
# print (inte)

#### Função para Extrair a Interseção entre duas Sentenças

In [2]:
# Função para realizar a interseção entre sentenças
def intersecao_string(X, Y):
    X = ' '.join(X.split())
    Y = ' '.join(Y.split())
    list_X = X.split(' ')
    list_Y = Y.split(' ')
    intersecao = ''
    i = 0
    for token in list_X:
        if token in list_Y:
            if i > 0:
                intersecao += ' '
            intersecao += token
        i += 1
    return intersecao.lower() # retorna a interseção em letras mínúsculas

#### Finção para fazer a União entre duas sentenças

In [3]:
# Função para realizar a união entre sentenças
def uniao_string(tupla_ann1, tupla_ann2):
    uniao = ''
    lst_sentenca1 = []
    lst_sentenca2 = []
    
    ann1 = {'inicio':tupla_ann1[1][0],'fim':tupla_ann1[1][1]}
    ann2 = {'inicio':tupla_ann2[1][0],'fim':tupla_ann2[1][1]}
    
    # determina o ínicio da sentença
    if ann1['inicio'] <= ann2['inicio']:
        inicio = ann1['inicio']
        sentenca1 = tupla_ann1[2]
        sentenca2 = tupla_ann2[2]
    else:
        inicio = ann2['inicio']
        sentenca1 = tupla_ann2[2]
        sentenca2 = tupla_ann1[2]
            
    # determina o fim da sentença
    if ann1['fim'] >= ann2['fim']:
        fim = ann1['fim']
    else:
        fim = ann2['fim']
    
    # verifica se uma sentença está contida na outra
    if ((ann1['fim'] <= ann2['fim']) and (ann1['inicio'] >= ann2['inicio'])):
        uniao = tupla_ann2[2]
        return inicio, fim, uniao.lower()
    else:
        if ((ann2['fim'] <= ann1['fim']) and (ann2['inicio'] >= ann1['inicio'])):
            uniao = tupla_ann1[2]
            return inicio, fim, uniao.lower()
        else:
            # Para o caso das sentenças não estarem contidas uma na outra    
            comprimento = fim - inicio
            diferenca   = abs(ann1['inicio'] - ann2['inicio'])
            
            idx = 0 - diferenca
            for i in range(comprimento):
                if i < len(sentenca1):
                    lst_sentenca1.append(sentenca1[i])
                else:
                    lst_sentenca1.append('~')
                    
                if i >= diferenca:
                    lst_sentenca2.append(sentenca2[idx])
                else:
                    lst_sentenca2.append('~')
                
                idx += 1

            for i in range(comprimento):
                if lst_sentenca1[i] != '~' and i < len(sentenca1):
                    uniao += lst_sentenca1[i]
                
                if i >= len(sentenca1) and lst_sentenca2[i] != '~':
                    uniao += lst_sentenca2[i]
                    
            return inicio, fim, uniao.lower() # retorna a interseção em letras mínúsculas

In [4]:
# # teste da funçao união
ann2 = ('ASSUNTO_CNJ', (70, 112), 'IPTU/ Imposto Predial e Territorial Urbano')
# ann1 = ('ASSUNTO_CNJ', (76, 122), 'Imposto Predial e Territorial Urbano nacional.')
ann1 = ('ASSUNTO_CNJ', (70, 112), 'IPTU/ Imposto Predial e Territorial Urbano')
uniao_string(ann1, ann2)

(70, 112, 'iptu/ imposto predial e territorial urbano')

#### Algoritmo principal de Cálculo das Métricas Kappa

In [5]:
import re
import time
import math
import glob, os, json
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import xlsxwriter
from tqdm import tqdm
import json

def get_dict_classe_tag(legenda):
    dict_classe_tag = dict()
    for tag in legenda.values():
        classe = tag
#         classe = tag.split("_")[0] # código original para criar os grupos de tags
        if classe not in dict_classe_tag.keys():
            dict_classe_tag[classe] = []
        dict_classe_tag[classe].append(tag)

    return dict_classe_tag

def inicializar_matriz_concordancia(dict_legenda):
    labels = []
    for chave, conjunto in dict_legenda.items():
        if conjunto == []:
            labels += [chave]
        else:
            labels += sorted(conjunto)
    labels = sorted(labels) + ['NENHUM']

    return pd.DataFrame(index=labels, columns=labels).fillna(0)

def combinacao(a, n):
    if n == 1:
        for x in a:
            yield [x]
    else:
        for i in range(len(a)):
            for x in combinacao(a[:i], n-1):
                yield [a[i]] + x
                

def similaridade_sentenca(X, Y):                # Program to measure the similarity between
                                              # two sentences using cosine similarity.
    # tokenization
    X_list = word_tokenize(X)
    X_list = [token.lower() for token in X_list]
    Y_list = word_tokenize(Y)
    Y_list = [token.lower() for token in Y_list]

    # sw contains the list of stopwords
    sw = stopwords.words('portuguese') 
    l1 =[];l2 =[]

    # remove stop words from the string
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw}

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0

    # cosine formula 
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    
    if cosine >= (1 - threshold):
        return cosine
    else:
        return 0              
                
def similaridade_jaccard(ann1, ann2):

    if ann1['inicio'] == -1 and ann2['inicio'] == -1:
        return 0
    else:
        intersecao = min(ann1['fim'], ann2['fim']) - max(ann1['inicio'], ann2['inicio'])
        uniao = max(ann1['fim'], ann2['fim']) - min(ann1['inicio'], ann2['inicio'])
        return max(intersecao/uniao,0)

def similaridade_esta_contido(ann1, ann2):
    if ((ann1['fim'] <= ann2['fim']) and (ann1['inicio'] >= ann2['inicio'])) or ((ann2['fim'] <= ann1['fim']) and (ann2['inicio'] >= ann1['inicio'])):
        return 1
    else:
        return 0

def similaridade_vizinhos(ann1, ann2, dist_max):
    fim =  min(ann1['fim'], ann2['fim'])
    inicio = max(ann1['inicio'], ann2['inicio'])

    if (inicio-fim) <= dist_max:
        return 1
    else:
        return 0

def medida_kappa(matriz_kappa, conjunto_tags=[]):
    p0 = 0  # É a concordância relativa observada entre avaliadores
    pe = 0  # É a probabilidade hipotética de concordância casual ("e"sperada).
    total = 2 * (matriz_kappa.sum().sum())

    if len(conjunto_tags) == 0:
          tags = intersecao_listas(matriz_kappa.index.values,
                             matriz_kappa.columns.values)
          tags.remove("NENHUM")

    for tag in tags:
        p0         += 2 * matriz_kappa.loc[tag, tag]     
        soma_linha  = matriz_kappa.loc[tag, :].sum()
        soma_coluna = matriz_kappa.loc[:, tag].sum()
        pe         += soma_linha*soma_coluna
            
    k = 0
    p0 = p0 / total
    pe = pe / (total*total)
    k = (p0-pe)/(1-pe)

    return(k)

def get_funcao_similaridade(threshold,*funcoes_similaridade):

    def funcao_similaridade(tupla_ann1,tupla_ann2):

        similaridades = []
        ann1 = {'inicio':tupla_ann1[1][0],'fim':tupla_ann1[1][1]}
        ann2 = {'inicio':tupla_ann2[1][0],'fim':tupla_ann2[1][1]}
        
        grupo_ann1 = tupla_ann1[0].split('_')
        grupo_ann2 = tupla_ann2[0].split('_')
        
        if grupo_ann1[0] == grupo_ann2[0]:      # Confere se os grupo da anotação é o mesmo
            
            if (tupla_ann1[0] == tupla_ann2[0]) and (tupla_ann1[0] == 'CLASSE_CNJ' or tupla_ann1[0] == 'ASSUNTO_CNJ'):
                ann1 = tupla_ann1[2]
                ann2 = tupla_ann2[2]
                similaridades.append(similaridade_sentenca(ann1,ann2))

                if max(similaridades) >= threshold:
                    return 1
                else:
                    return 0
            
            else:
                if (tupla_ann1[0] == tupla_ann2[0]):
                    lista_funcoes = [similaridade_jaccard, similaridade_esta_contido]
                    similaridades  = [f(ann1, ann2) for f in funcoes_similaridade ]
                    if max(similaridades) >= threshold:
                        return 1
                    else:
                        return 0
                else:
                    return float('nan')

        else:
            return float("nan")
        
    return funcao_similaridade

def intersecao_listas(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def listas_anotacoes_para_matriz_concordancia(lista_anotacoes_1,lista_anotacoes_2,legenda):

    matriz_concordancia = inicializar_matriz_concordancia(dict_entidades)

    indice = lista_anotacoes_1 if len(lista_anotacoes_1) > 0 else np.array([['nada',(-1,-1)]])
    col = lista_anotacoes_2 if len(lista_anotacoes_2) > 0 else np.array([['nada',(-1,-1)]])
    matriz_anotacao = pd.DataFrame(index=indice, columns=col).fillna(0)              
    df = matriz_anotacao.stack()
    
    # Lista de tags e anotações para a CDJUR 
    tag_ann = []
    
    # Lista de tags discordantes
    tag_ann_disc = []
    ct_app=0
    for linha in df.keys():
        un_string = ''
        ann1 = linha[0]
        ann2 = linha[1]
        df.loc[ann1, ann2] = funcao_similaridade(ann1, ann2)

#         print('-------------------------------------------------')
        
        if df.loc[ann1, ann2] == 1 and ann1[0] != 'nada':
#             print('**************************** há concordância *************************')
#             tag_ann.append((ann1[0], intersecao_string(ann1[2], ann2[2]))) # guarda a anotação por interseção
            un_string = uniao_string(ann1, ann2)
#             print('ann1', ann1[0], ann1[1][0], ann1[1][1])
#             print(tag_ann)
            # verifica se há erro de tag repetidas (anotação duplicada)
            if  (ann1[0], un_string) not in tag_ann:
                if len(tag_ann) == 0:
#                     print('tag_ann vazia - append')
                    tag_ann.append((ann1[0], un_string)) # guarda a anotação por união
                    ct_app += 1
                else:
                    # confere se há sobreposição com as tag já guardadas
                    for annot in tag_ann:
                        intersecao = False
                        intersecao = getOverlap([ann1[1][0], ann1[1][1]],[annot[1][0], annot[1][1]])
                        if intersecao and ann1[0] == annot[0]:
#                             print(ann1[0], [ann1[1][0], ann1[1][1]],' - ', [annot[1][0], annot[1][1]])
#                             print('ann1', ann1[2])
#                             print('annot', annot[1][2])
#                             prin
                            ann2_aux = (annot[0], (annot[1][0], annot[1][1]),annot[1][2])
                            new_annot = (annot[0], (uniao_string(ann1, ann2_aux)))
                            annot = new_annot
                            break
                    
                    ####
                    if intersecao and ann1[0] == annot[0]:
                        ann2_aux = (annot[0], (annot[1][0], annot[1][1]),annot[1][2])
                        new_annot = (annot[0], (uniao_string(ann1, ann2_aux)))
                        annot = new_annot
#                         print('tag repetida revisada',ann1[0], [ann1[1][0], ann1[1][1]],' - ', annot[0],[annot[1][0], annot[1][1]])
                    else:
                        if ann1[0] == annot[0] and not intersecao:
#                             print(intersecao, 'append tag_ann', ann1[0])
                            tag_ann.append((ann1[0], un_string)) # guarda a anotação por união
                            ct_app += 1
                        if ann1[0] != annot[0] and not intersecao:
#                             print(intersecao, 'append tag_ann', ann1[0])
                            tag_ann.append((ann1[0], un_string)) # guarda a anotação por união
                            ct_app += 1
                    ####

        else:
            if df.loc[ann1, ann2] == 0 and ann1[0] != 'nada':
                intersecao = False
                intersecao = getOverlap([ann1[1][0], ann1[1][1]],[ann2[1][0], ann2[1][1]])
                if intersecao:
                    # registra anotações discordantes
                    un_string = uniao_string(ann1, ann2)
                    # verifica se há erro de tag repetidas (anotação duplicada)
                    if  (ann1[0], un_string) not in tag_ann:
                        tag_ann_disc.append((ann1[0], un_string)) # guarda a anotação por união
     
    matriz_anotacao = df.unstack()

    linha_nenhum = pd.Series(matriz_anotacao.sum(axis=0).apply(lambda x: 0 if x >= 1 or not math.isnan(x) else 1), name='NENHUM')
    matriz_anotacao = matriz_anotacao.append(linha_nenhum)
    matriz_anotacao['NENHUM'] = matriz_anotacao.sum(axis=1).apply(lambda x: 0 if x>=1 else 1)
    matriz_anotacao.loc['NENHUM','NENHUM'] = 0
    matriz_anotacao.index = [ind[0] for ind in matriz_anotacao.index[:-1]] + ['NENHUM']
    matriz_anotacao.columns = [ind[0] for ind in matriz_anotacao.columns[:-1]] + ['NENHUM']
    matriz_anotacao = matriz_anotacao.groupby(matriz_anotacao.columns, axis=1).sum().groupby(matriz_anotacao.index, axis=0).sum()
    
    matriz_concordancia.update(matriz_anotacao)
    
    return matriz_concordancia, tag_ann, tag_ann_disc, ct_app

def ler_anotacoes(caminho_ann):

    lista = []
    anotacoes = json.load(open(caminho_ann,'rb'))
    
    # Inclui informações para as tags Classe e Assunto CNJ (m_68 e m_69)
    if len(anotacoes['metas']) == 1:
        try:
            tag = 'CLASSE_CNJ'
            inicio = 1   #apenas uma constante para atender ao algoritmo já existente
            texto  = anotacoes['metas']['m_68']['value']
            fim = inicio+len(texto)
            lista.append((tag,(inicio,fim),texto))
        except:
            tag = 'ASSUNTO_CNJ'
            inicio = 50   #apenas uma constante para atender ao algoritmo já existente
            texto  = anotacoes['metas']['m_69']['value']
            fim = inicio+len(texto)
            lista.append((tag,(inicio,fim),texto))
            
    if len(anotacoes['metas']) == 2:
        tag = 'CLASSE_CNJ'
        inicio = 1   #apenas uma constante para atender ao algoritmo já existente
        texto  = anotacoes['metas']['m_68']['value']
        fim = inicio+len(texto)
        lista.append((tag,(inicio,fim),texto))
        
        tag = 'ASSUNTO_CNJ'
        inicio = 50   #apenas uma constante para atender ao algoritmo já existente
        texto  = anotacoes['metas']['m_69']['value']
        fim = inicio+len(texto)
        lista.append((tag,(inicio,fim),texto))


    for ann in anotacoes['entities']:
        
        tag = legenda_cd[ann['classId']]
        inicio = ann['offsets'][0]['start']
        texto = ann['offsets'][0]['text']
        fim = inicio+len(texto)
        
        lista.append((tag,(inicio,fim),texto))

    return lista

def get_submatriz_concordancia(matriz,lista_tags):
    resultado = matriz.copy()
    resultado = resultado.loc[lista_tags+['NENHUM'],lista_tags+['NENHUM']]
    

    resultado.loc['NENHUM',lista_tags] += matriz.loc[~matriz.index.isin(resultado.index),matriz.columns.isin(lista_tags)].sum(axis=0).loc[lista_tags]
    resultado.loc[lista_tags,"NENHUM"] += matriz.loc[matriz.index.isin(lista_tags),~matriz.columns.isin(resultado.columns),].sum(axis=1).loc[lista_tags]
    return resultado

# PRIMEIRA ETAPA - Cria um dataframe dos documentos                                                PRIMEIRA ETAPA
def caminho_anotacoes_para_df(caminho_cdjur,legenda):
    print('Iniciando primeira etapa.')
#     print(caminho_cdjur)
    lista_docs = glob.glob(caminho_cdjur + "/**/*.json",recursive=True)
    lista_documentos = []
    for doc_completo in tqdm(lista_docs):
        d = {}
        doc = doc_completo.replace(caminho_cdjur,"").split(os.sep)
        anotador = doc[1]
#         cddocumento = doc[-1].split("_")[-1].split(".")[0] # original
        cddocumento = doc[-1].split("-")[-1].split(".")[0] # versão com o nome completo do arquivo
        d['anotador'] = anotador
        d['cddocumento'] = cddocumento
        d['caminho'] = doc_completo
        lista_documentos.append(d)

    dataframe_documentos = pd.DataFrame(lista_documentos).groupby('cddocumento').agg(list).reset_index()
    dataframe_documentos['anotacoes'] = dataframe_documentos['caminho'].apply(lambda x: [ler_anotacoes(y) for y in x])
    dataframe_documentos.to_csv('df_docs'+sufixo_file+'.csv', sep=',', encoding='utf-8') #Dataframe de ocorrência de tags
    dataframe_documentos = dataframe_documentos.drop('caminho',axis=1)
    lista_matrizes   = []
    lista_cdjur      = []
    lista_cdjur_disc = []
    dicionario_cdjur      = {} # Cria o dicionário final da cdjur por documento
    dicionario_cdjur_disc = {} # Cria o dicionário final da cdjur por documento - discordante

    print('Iniciando segunda etapa.')
    conta_append = 0
    conta = 0
    ct_tag = 0
    for documento in tqdm(dataframe_documentos.itertuples(index=True)):
        dict_doc        = {}
        dict_doc_disc   = {}
        lista_dict      = []
        lista_dict_disc = []
        # Lista de tag_anotação de cada documento
        tag_anotacao   = []
        # Lista de anotações discordantes
        tag_anotacao_disc   = []
        anotadores = documento.anotador

        if len(anotadores) > 2:
            print(documento.cddocumento)
            print("Nr Par anotadores:", anotadores)
            print(anotadores[0], anotadores[1])
            anotadores = anotadores[:-1]
        
        par_anotadores = combinacao(anotadores,2)
        dict_matriz_doc = dict()
            
        for anotador1,anotador2 in par_anotadores:
            dict_matriz_doc = dict()
            dict_matriz_doc['cddocumento'] = documento.cddocumento
            dict_matriz_doc['anotador1'] = anotador1
            dict_matriz_doc['anotador2'] = anotador2
            id1, = np.where(np.array(anotadores) == anotador1)[0]
            id2, = np.where(np.array(anotadores) == anotador2)[0]
            lista_anotacoes_1 = pd.DataFrame(np.array(sorted(documento.anotacoes[id1],key= lambda x: x[0]))).drop_duplicates().values
            lista_anotacoes_2 = pd.DataFrame(np.array(sorted(documento.anotacoes[id2],key= lambda x: x[0]))).drop_duplicates().values
            matriz_concordancia, tag_anotacao, tag_anotacao_disc,conta_append = listas_anotacoes_para_matriz_concordancia(lista_anotacoes_1,lista_anotacoes_2,legenda)
            dict_matriz_doc['matriz_concordancia'] = matriz_concordancia
            conta += conta_append
            lista_matrizes.append(dict_matriz_doc)
            dict_doc      = {}
            dict_doc_disc = {}
            
            # cria uma lista com as anotações de todos os documentos
            ct_tag += len(tag_anotacao)
            for item in tag_anotacao:
                lista_cdjur.append(item)
                if item[0] == 'CLASSECNJ' or item[0] == 'ASSUNTOCNJ':
                    pass
                else:
                    dict_doc[item[0]] = {'inicio': item[1][0], 'fim': item[1][1], 'texto': item[1][2]}
                    lista_dict.append(dict_doc)
                    dict_doc = {}

            # cria uma lista com as anotações de todos os documentos - discordantes
            for item in tag_anotacao_disc:
                lista_cdjur_disc.append(item)
                if item[0] == 'CLASSECNJ' or item[0] == 'ASSUNTOCNJ':
                    pass
                else:
                    dict_doc_disc[item[0]] = {'inicio': item[1][0], 'fim': item[1][1], 'texto': item[1][2]}
                    lista_dict_disc.append(dict_doc_disc)
                    dict_doc_disc = {}

        # Cria o dicionário final da cdjur por documento
        dicionario_cdjur[documento.cddocumento] = lista_dict
        lista_dict      = []
        # Cria o dicionário final da cdjur por documento - discordante
        dicionario_cdjur_disc[documento.cddocumento] = lista_dict_disc
        lista_dict_disc = []
        
    print("conta lista_anotacao", ct_tag)
    print("conta appends", conta)
    print('Quantidade de anotações:', len(lista_cdjur))
    print('Quantidade de anotações discordantes:', len(lista_cdjur_disc))
    
    # Retira as repetições de anotações (deixa lista única)
    lst_cdjur = list(set(lista_cdjur))
    
    # Cria dicionário para a CDJUR
    # Seleciona as tags para usar como chaves no dicionário
    chaves = []
    for idx,tupla in enumerate(lst_cdjur):
        chaves.append(tupla[0])

    dict_cdjur = dict.fromkeys(list(set(chaves)), [])

    for tag in dict_cdjur.keys():
        lista_aux = []
            
        for tag_ann in lista_cdjur:
            if tag_ann[0] == tag:
                lista_aux.append(tag_ann[1][2])

        dict_cdjur[tag] = lista_aux

    with open('cdjur'+sufixo_file+'.json', 'w', encoding='utf-8') as file:
        json.dump(dict_cdjur, file, ensure_ascii=False)
    
###### para gerar lista de tags discordantes

    # Retira as repetições de anotações (deixa lista única)                 
    lst_cdjur_disc = list(set(lista_cdjur_disc))
    
    # Cria dicionário para a CDJUR
    # Seleciona as tags para usar como chaves no dicionário
    chaves = []
    for idx,tupla in enumerate(lst_cdjur_disc):
        chaves.append(tupla[0])

    dict_cdjur_disc = dict.fromkeys(list(set(chaves)), [])

    for tag in dict_cdjur_disc.keys():
        lista_aux = []
            
        for tag_ann in lista_cdjur_disc:
            if tag_ann[0] == tag:
                lista_aux.append(tag_ann[1][2])

        dict_cdjur_disc[tag] = lista_aux

    with open('cdjur_disc'+sufixo_file+'.json', 'w', encoding='utf-8') as file:
        json.dump(dict_cdjur_disc, file, ensure_ascii=False)
######
        
#   grava arquivo final da cdjur por documento
    with open('cdjur_por_documento'+sufixo_file+'.json', 'w', encoding='utf-8') as file:
        json.dump(dicionario_cdjur, file, ensure_ascii=False)

#   grava arquivo final da cdjur por documento - discordantes
    with open('cdjur_por_documento_disc'+sufixo_file+'.json', 'w', encoding='utf-8') as file:
        json.dump(dicionario_cdjur_disc, file, ensure_ascii=False)
    
    dataframe_documentos = pd.DataFrame(lista_matrizes)
    
    # Gera Dataframe da Coleção Dourada e grava em arquivo .csv
#     df_cdjur = dataframe_documentos[['cddocumento', 'tag_anotacao']]
#     df_cdjur.drop(['Unnamed: 0'], axis=1, inplace=True)
#     df_cdjur.to_csv('cdjur'+sufixo_file+'.csv', sep=',', encoding='utf-8')

    return dataframe_documentos

# ETAPA FINAL - Gera as planilhas com os valores Kappa                                                 ETAPA FINAL
def dataframe_documentos_para_xlsx(dataframe_documentos, dict_entidades, caminho_xls):
    print('Iniciando última etapa.')
    dataframe_documentos['kappa_geral'] = dataframe_documentos['matriz_concordancia'].apply(medida_kappa)
    for categoria, tags in tqdm(dict_entidades.items()):
        dataframe_documentos['kappa_'+categoria.lower()] = dataframe_documentos['matriz_concordancia'].apply(lambda x: get_submatriz_concordancia(x,tags)).apply(medida_kappa)
    
    dataframe_documentos.to_csv('df_doc_completo'+sufixo_file+'.csv', sep=',', encoding='utf-8') # Provisório
    writer = pd.ExcelWriter(caminho_xls, engine='xlsxwriter')
    dataframe_documentos.drop('matriz_concordancia',axis=1).to_excel(writer,sheet_name= 'resumo_kappa',index=True,startcol=0,startrow=0)

    for linha in tqdm(dataframe_documentos.itertuples(index=True)):
        startcol=0
        startrow=0
        sheet_name = linha.Index
        matriz_concordancia = linha.matriz_concordancia
        matriz_concordancia.replace(0,"-").to_excel(writer,sheet_name= str(sheet_name),index=True,startcol=startcol,startrow=startrow)
        startrow += matriz_concordancia.shape[0] + 2
        for categoria, tags in dict_entidades.items():
            m = get_submatriz_concordancia(matriz_concordancia,tags)
            m.replace(0,"-").to_excel(writer,sheet_name= str(sheet_name),index=True,startcol=startcol,startrow=startrow)
            startrow += m.shape[0] + 2

    writer.close()

#                                       MÓDULO INICIAL DE CÁLCULO DAS MÉTRICAS
if __name__ == "__main__":
#     del str
    start_time = time.time()
    threshold = 0.1
    funcao_similaridade = get_funcao_similaridade(threshold, similaridade_jaccard,similaridade_esta_contido)
    
#     legenda_cd = {'e_71': 'PES_PROMOTOR_MP', 'e_60': 'PROVA_ACAREAÇÃO', 'e_16': 'PES_REU_JUR', 'e_35': 'PENA_RESTRITIVA_DIREITO',
#                   'e_64': 'PROVA_DECL_VITIMA', 'e_38': 'END_REU', 'e_53': 'PROVA_INSTRUM_CRIME', 'e_34': 'PENA_MUL',
#                   'e_56': 'PROVA_CONFISSÃO', 'e_67': 'END_OUTROS', 'e_30': 'SENTENÇA_SEM_RESOL', 'e_12': 'PES_AUTOR_JUR',
#                   'e_23': 'PES_TESTEMUNHA', 'e_70': 'END_VITIMA', 'e_52': 'PROVA_LOCAL_CRIME', 'e_63': 'PROVA_DISPMOVEIS',
#                   'e_31': 'SENTENÇA_ACORDO', 'e_20': 'PES_ADVOG_AUT', 'e_18': 'PES_JUIZ', 'e_29': 'SENTENÇA_IMPROCED',
#                   'e_57': 'PROVA_TESTEM_ACUS', 'e_11': 'PES_AUTOR_FIS', 'e_58': 'PROVA_TESTEM_DEFESA', 'e_65': 'PROVA_OBJETO_APREENDIDO',
#                   'e_33': 'PENA_INDENIZACAO', 'e_54': 'PROVA_EXAME_LAB', 'e_48': 'PROVA_CORPO_DELITO', 'e_19': 'PES_JURI',
#                   'e_51': 'PROVA_EXUMAÇÃO', 'e_37': 'END_AUTOR', 'e_62': 'PROVA_REPRODUÇÃO', 'm_69': 'ASSUNTO_CNJ',
#                   'e_73': 'SENTENÇA_PROCED_PARCIAL', 'e_26': 'PES_OUTROS', 'e_39': 'END_DELITO', 'e_40': 'END_TESTEMUNHA', 'e_21': 'PES_ADVOG_REU',
#                   'e_59': 'PROVA_RECONHECIMENTO', 'e_28': 'SENTENÇA_PROCED', 'e_17': 'PES_VITIMA', 'e_9': 'NOR_PRINCIPAL',
#                   'e_10': 'NOR_ACESSORIA', 'e_55': 'PROVA_INTERROG', 'm_68': 'CLASSE_CNJ', 'e_32': 'DEC_DECISÃO_INTERL',
#                   'e_61': 'PROVA_DOCUMENTAL', 'e_36': 'PENA_PRISAO', 'e_50': 'PROVA_AUTOPSIA', 'e_25': 'PES_AUTORID_POLICIAL',
#                   'e_24': 'PES_MP_CUSTOS_IURIS', 'e_13': 'PES_REU_FIS', 'e_72': 'NOR_JURISPRUDÊNCIA'}
#     print('Gerando Coleção Dourada do Ministério Público.')
#     caminho_cdjur_mp = "C://Users//mbrit/tagtog_CDJUR-MP/CDJUR-MP/ann.json/master"
#     caminho_cdjur_mp = "C://Users//mbrit/tagtog_CDJUR-MP/CDJUR-MP/ann.json/members"
#     caminho_legenda_mp = "C://Users//mbrit/tagtog_CDJUR-MP/CDJUR-MP/annotations-legend.json"
#     sufixo_file = '_rev_mp'
#     legenda_cd = json.load(open(caminho_legenda_mp,'rb'))
#     legenda_cd = {key: value for key, value in legenda_cd.items() if key.startswith("e_") or key.startswith("m_")}
    
#     dict_entidades = get_dict_classe_tag(legenda_cd)
#     dataframe_documentos = caminho_anotacoes_para_df(caminho_cdjur_mp,legenda_cd)    
#     dataframe_documentos_para_xlsx(dataframe_documentos, dict_entidades, "metrica"+sufixo_file+".xlsx")

#     Início TJ
    print('Gerando Coleção Dourada do Tribunal de Justiça.')
    caminho_cdjur_tj = "C://Users//mbrit/tagtog_CDJUR/CDJUR/ann.json/master"
#     caminho_cdjur_tj = "C://Users//mbrit/tagtog_CDJUR/CDJUR/ann.json/members" # original
    caminho_legenda_tj = "C://Users//mbrit/tagtog_CDJUR/CDJUR/annotations-legend.json"
    sufixo_file = '_master_tj'
    
    legenda_cd = json.load(open(caminho_legenda_tj,'rb'))
    
    legenda_cd = {key: value for key, value in legenda_cd.items() if key.startswith("e_") or key.startswith("m_")}
    
    dict_entidades = get_dict_classe_tag(legenda_cd)

    dataframe_documentos = caminho_anotacoes_para_df(caminho_cdjur_tj,legenda_cd)
#     dataframe_documentos_para_xlsx(dataframe_documentos, dict_entidades, "metrica"+sufixo_file+".xlsx")
    
    import winsound
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    duration = 900  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    print("--- %s segundos ---" % (time.time() - start_time))

Gerando Coleção Dourada do Tribunal de Justiça.
Iniciando primeira etapa.


100%|██████████████████████████████████████████████████████████████████████████| 1240/1240 [00:00<00:00, 310077.92it/s]


Iniciando segunda etapa.


0it [00:00, ?it/s]C:\Users\mbrit\anaconda3\envs\projeto_tj_mp\lib\site-packages\ipykernel_launcher.py:374: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\mbrit\anaconda3\envs\projeto_tj_mp\lib\site-packages\ipykernel_launcher.py:375: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
134it [00:10,  7.64it/s]C:\Users\mbrit\anaconda3\envs\projeto_tj_mp\lib\site-packages\ipykernel_launcher.py:174: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or sh

conta lista_anotacao 13407
conta appends 13407
Quantidade de anotações: 13407
Quantidade de anotações discordantes: 2
--- 169.77430248260498 segundos ---


### Módulo de Estatísticas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_com = pd.read_csv('df_doc_completo_mp.csv')
df_com.tail(5)

In [ ]:
df_com.loc[594,"matriz_concordancia"]

### Carrega Dataset de documentos e anotações

In [ ]:
df_docs = pd.read_csv('df_docs_int_mp.csv')

In [ ]:
df_docs.head(7)

### Pré-processamento para gerar estatísticas das tags

In [ ]:
set_tag = ['CLASSE_CNJ', 'ASSUNTO_CNJ', 'NOR_PRINCIPAL', 'NOR_ACESSORIA', 'NOR_JURISPRUDÊNCIA', 'PES_VITIMA', 'PES_AUTOR_FIS', 'PES_AUTOR_JUR', 'PES_REU_FIS', 'PES_REU_JUR', 'PES_JUIZ',
'PES_JURI', 'PES_ADVOG_AUT', 'PES_ADVOG_REU', 'PES_TESTEMUNHA', 'PES_MP_CUSTOS_IURIS', 'PES_AUTORID_POLICIAL', 'PES_OUTROS',
'PES_PROMOTOR_MP', 'SENTENÇA_PROCED', 'SENTENÇA_IMPROCED', 'SENTENÇA_PROCED_PARCIAL', 'SENTENÇA_SEM_RESOL', 'SENTENÇA_ACORDO', 'DEC_DECISÃO_INTERL', 'PENA_INDENIZACAO',
'PENA_MUL', 'PENA_RESTRITIVA_DIREITO', 'PENA_PRISAO', 'END_AUTOR', 'END_REU', 'END_DELITO', 'END_TESTEMUNHA','END_OUTROS',
'END_VITIMA', 'PROVA_CORPO_DELITO', 'PROVA_AUTOPSIA', 'PROVA_EXUMAÇÃO', 'PROVA_LOCAL_CRIME', 'PROVA_INSTRUM_CRIME', 'PROVA_EXAME_LAB',
'PROVA_INTERROG', 'PROVA_CONFISSÃO', 'PROVA_TESTEM_ACUS', 'PROVA_TESTEM_DEFESA', 'PROVA_RECONHECIMENTO', 'PROVA_ACAREAÇÃO',
'PROVA_DOCUMENTAL', 'PROVA_REPRODUÇÃO', 'PROVA_DISPMOVEIS', 'PROVA_DECL_VITIMA', 'PROVA_OBJETO_APREENDIDO']

In [ ]:
import re

lista_tag = [] # lista geral de ocorrências de tags em todos os documentos
# set_tag = []
# df_estat_tag = pd.DataFrame()
for lista in df_docs["anotacoes"].values:
    tokens = re.split(r'\W+', lista)
    lista_doc_tag = [] # Lista de tags por documento
    for tk in tokens:
        if tk in set_tag:
            tag = tk.split()
            try:
#             if not tag[0].isdigit():
                lista_doc_tag.append(tag[0])
#                 set_tag.append(tag[0])
            except:
                pass

    lista_tag.append(lista_doc_tag)

# set_tag = list(set(set_tag)) # Conjunto de tags (únicas)

### Cria Dataset de estatística das tags

In [ ]:
# Colunas do novo dataset
columns_nm = ['cddocumento','tags'] + set_tag
df_estat_tag = pd.DataFrame(columns=columns_nm)
df_estat_tag['cddocumento'] = df_docs['cddocumento']
df_estat_tag['tags'] = lista_tag

### Totaliza ocorrência de tags por documento

In [ ]:
estatistica = {}
for tag in set_tag:
    estatistica[tag] = 0
for i, lista in enumerate(df_estat_tag['tags'].values):
    for tag in lista:
        estatistica[tag] = estatistica[tag] + 1
    for tag in set_tag:
        df_estat_tag.loc[i,tag] = estatistica[tag]
        estatistica[tag] = 0

### Total de anotações por tag

In [ ]:
# Obtenha um novo conjunto com elementos que estão apenas em a, mas não em b
cols = []
cols = [x for x in set_tag] # if (x not in ['PES_TESTEMUNHA_AUT'])]
# Grava csv
df_estat_tag[cols].sum().sort_values().to_csv('sumário_tags_mp.csv', sep=',', encoding='utf-8')
df_estat_tag[cols].sum()

In [ ]:
df_estat_tag[cols].sum().sum()

In [ ]:
plt.figure(figsize=(6,14))
df_estat_tag[cols].sum().plot.barh(title="Quantidade de Anotações por Tag")

In [ ]:
# Gera dataset com colunas selecionadas
cols = ['cddocumento'] + cols
df = df_estat_tag[cols].copy()

In [ ]:
df.shape

In [ ]:
df.head()

### Grava arquivo de estatístivas das tags

In [ ]:
# Grava arquivo de estatísticas
df.to_csv('estatisticas_tags_mp.csv', sep=',', encoding='utf-8')

### Abre a Coleção Dourada

In [ ]:
df_cdjur = pd.read_csv('cdjur.csv')
df_cdjur.head()

In [ ]:
df_cdjur.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
pd.set_option('max_colwidth', 800)
df_cdjur.head()

### Abre CDJUR JSON - Discordantes

In [ ]:
# Abre CDJUR do TJ + Revisada
cdjur_an_tj = open('cdjur_disc_an_tj.json', "r")
cdjur_rev_tj = open('cdjur_disc_rev_tj.json', "r")

# Abre CDJUR do MP + Revisada
cdjur_an_mp = open('cdjur_disc_an_mp.json', "r")
cdjur_rev_mp = open('cdjur_disc_rev_mp.json', "r")

# Retorna CDJUR do TJ como Dicionário do Python
dict_cdjur_an_tj = json.load(cdjur_an_tj)
dict_cdjur_rev_tj = json.load(cdjur_rev_tj)

# Retorna CDJUR do MP como Dicionário do Python
dict_cdjur_an_mp = json.load(cdjur_an_mp)
dict_cdjur_rev_mp = json.load(cdjur_rev_mp)

In [ ]:
# TJ Anotações discordantes
a = 0 
for tag in dict_cdjur_an_tj.keys():
    a += len(dict_cdjur_an_tj[tag])
    
# TJ Revisões discordantes
b = 0
for tag in dict_cdjur_rev_tj.keys():
    b += len(dict_cdjur_rev_tj[tag])
    
# MP Anotações discordantes
c = 0
for tag in dict_cdjur_an_mp.keys():
    c += len(dict_cdjur_an_mp[tag])
    
# MP Revisões discordantes
d = 0
for tag in dict_cdjur_rev_mp.keys():
    d += len(dict_cdjur_rev_mp[tag])

print(f'TJ - Tags:{len(dict_cdjur_an_tj)}, Anotações:{a}')
print(f'TJ - Tags:{len(dict_cdjur_rev_tj)}, Anotações:{b}')
print(f'Total TJ: {a+b}')
print(f'MP - Tags:{len(dict_cdjur_an_mp)}, Anotações:{c}')
print(f'MP - Tags:{len(dict_cdjur_rev_mp)}, Anotações:{d}')
print(f'Total MP: {c+d}')

In [ ]:
# Exclue tags inválidas
# cdjur_mp
del dict_cdjur_an_mp['CLASSECNJ']
del dict_cdjur_an_mp['ASSUNTOCNJ']
# cdjur_rev_mp
del dict_cdjur_rev_mp['CLASSECNJ']
del dict_cdjur_rev_mp['ASSUNTOCNJ']

In [ ]:
dict_cdjur_an_tj

### Juntas as Coleções Douradas - Interseção

In [ ]:
# junta as coleçções do TJ
dict_cdjur1 = dict_cdjur_int_tj
for i, tag2 in enumerate(dict_cdjur_rev_int_tj):
    if tag2 in dict_cdjur_int_tj.keys():
        dict_cdjur1[tag2] = list(set(dict_cdjur_int_tj[tag2]+dict_cdjur_rev_int_tj[tag2]))
    else:
        dict_cdjur1[tag2] = dict_cdjur_rev_int_tj[tag2]
# junta as coleçções do MP
dict_cdjur2 = dict_cdjur_int_mp
for i, tag2 in enumerate(dict_cdjur_rev_int_mp):
    if tag2 in dict_cdjur_int_mp.keys():
        dict_cdjur2[tag2] = list(set(dict_cdjur_int_mp[tag2]+dict_cdjur_rev_int_mp[tag2]))
    else:
        dict_cdjur2[tag2] = dict_cdjur_rev_int_mp[tag2]
# junta as coleções do TJ e MP
dict_cdjur_int = dict_cdjur1
for i, tag2 in enumerate(dict_cdjur2):
    if tag2 in dict_cdjur1.keys():
        dict_cdjur_int[tag2] = list(set(dict_cdjur1[tag2]+dict_cdjur2[tag2]))
    else:
        dict_cdjur_int[tag2] = dict_cdjur2[tag2]

#### Grava Coleção Dourada Única

In [ ]:
with open('cdjur_int.json', 'w', encoding='utf-8') as file:
    json.dump(dict_cdjur_int, file)

# Fecha os arquivos
cdjur_int_tj.close()
cdjur_rev_int_tj.close()
cdjur_int_mp.close()
cdjur_rev_int_mp.close()

#### Visualização da CDJUR

In [ ]:
len(dict_cdjur_int)

#### Descritivo

In [ ]:
print('Tag:    Quantidade de Anotações:')
total = 0
quantidades = []
for tag in dict_cdjur_int.keys():
    print(tag, len(dict_cdjur_int[tag]))
    total += len(dict_cdjur_int[tag])
    quantidades.append(len(dict_cdjur_int[tag]))
print('\n Total:', total)

#### Gráfico de Barras

In [ ]:
tags = list(dict_cdjur_int.keys())
plt.figure(figsize=(10,14))
plt.barh(range(len(dict_cdjur_int)), quantidades, tick_label=tags)
plt.title("Ocorrências por Tag")

plt.grid()
plt.show()

#### Algumas amostras da CDJUR

In [ ]:
len(dict_cdjur_int['SENTENÇA_PROCED_PARCIAL'])

In [ ]:
dict_cdjur_int['PES_MP_CUSTOS_IURIS']

In [ ]:
dict_cdjur_int['PES_JURI']

### Juntas as Coleções Douradas - União

In [ ]:
# junta as coleções do TJ
dict_cdjur1 = dict_cdjur_un_tj
for i, tag2 in enumerate(dict_cdjur_rev_un_tj):
    if tag2 in dict_cdjur_un_tj.keys():
        dict_cdjur1[tag2] = list(set(dict_cdjur_un_tj[tag2]+dict_cdjur_rev_un_tj[tag2]))
    else:
        dict_cdjur1[tag2] = dict_cdjur_rev_un_tj[tag2]
# junta as coleçções do MP
dict_cdjur2 = dict_cdjur_un_mp
for i, tag2 in enumerate(dict_cdjur_rev_un_mp):
    if tag2 in dict_cdjur_un_mp.keys():
        dict_cdjur2[tag2] = list(set(dict_cdjur_un_mp[tag2]+dict_cdjur_rev_un_mp[tag2]))
    else:
        dict_cdjur2[tag2] = dict_cdjur_rev_un_mp[tag2]
# junta as coleções do TJ e MP
dict_cdjur_un = dict_cdjur1
for i, tag2 in enumerate(dict_cdjur2):
    if tag2 in dict_cdjur1.keys():
        dict_cdjur_un[tag2] = list(set(dict_cdjur1[tag2]+dict_cdjur2[tag2]))
    else:
        dict_cdjur_un[tag2] = dict_cdjur2[tag2]

#### Grava CDJUR única - União

In [ ]:
with open('cdjur_un.json', 'w', encoding='utf-8') as file:
    json.dump(dict_cdjur_un, file)

# Fecha os arquivos
cdjur_un_tj.close()
cdjur_rev_un_tj.close()
cdjur_un_mp.close()
cdjur_rev_un_mp.close()

#### Visualização da CDJUR

In [ ]:
len(dict_cdjur_un)

#### Descritivo

In [ ]:
print('Tag:    Quantidade de Anotações:')
total = 0
quantidades = []
for tag in dict_cdjur_un.keys():
    print(tag, len(dict_cdjur_un[tag]))
    total += len(dict_cdjur_un[tag])
    quantidades.append(len(dict_cdjur_un[tag]))
print('\n Total:', total)

#### Gráfico de Barras

In [ ]:
tags = list(dict_cdjur_un.keys())
plt.figure(figsize=(10,14))
plt.barh(range(len(dict_cdjur_un)), quantidades, tick_label=tags)
plt.title("Ocorrências por Tag")

plt.grid()
plt.show()

#### Algumas amostras da CDJUR

In [ ]:
len(dict_cdjur_un['SENTENÇA_PROCED_PARCIAL'])

In [ ]:
dict_cdjur_un['SENTENÇA_ACORDO']

In [ ]:
dict_cdjur_un['PROVA_LOCAL_CRIME']